In [ ]:
https://github.com/vincentropy/python_cartography_tutorial/blob/master/part1_basic_folium_maps.ipynb

In [ ]:
https://github.com/vincentropy/python_cartography_tutorial/blob/master/part2_custom_layers.ipynb

In [ ]:
https://github.com/vincentropy/python_cartography_tutorial/blob/master/part3_animations.ipynb

In [1]:
import pandas as pd
import folium

In [2]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

folium.CircleMarker(location=[40.738, -73.98],fill=True).add_to(folium_map)
folium_map

In [3]:
bike_data = pd.read_csv("D:\Eigene Dateien\OneDrive\Dokumente\GitHub\PDS_Project/nextbike/data/output/Trips.csv")
bike_data["Start_Time"] = pd.to_datetime(bike_data["Start_Time"])
bike_data["End_Time"] = pd.to_datetime(bike_data["End_Time"])
bike_data["hour"] = bike_data["Start_Time"].map(lambda x: x.hour)

In [4]:
bike_data.head(6)

,index,p_spot_start,p_place_type_start,Start_Time,Bike_Number,p_uid_start,p_bikes_start,Latitude_start,b_bike_type_start,Place_start,...,Day_end,Hour_end,Minute_end,Day_of_year_end,Postalcode_start,Postalcode_end,Dist_start,Dist_end,Direction,hour
0,0,True,0,2019-03-20 13:56:00,90000,66128,5,49.435580,71,Hst. Dutzendteich,...,20,14,56,79,90478,90478,3.260892,2.880538,True,13
1,1,True,0,2019-12-15 23:07:00,90001,66128,6,49.435605,71,Dutzendteich,...,15,23,7,349,90478,90478,3.261340,3.261340,False,23
2,2,True,0,2019-12-17 16:46:00,90001,66128,6,49.435605,71,Dutzendteich,...,17,16,46,351,90478,90478,3.261340,3.261340,False,16
3,3,True,0,2019-02-22 07:31:00,90004,66126,5,49.433693,4,Doku-Zentrum,...,22,7,31,53,90478,90478,3.066872,3.066872,False,7
4,4,True,0,2019-03-04 16:32:00,90004,66126,5,49.433693,4,Doku-Zentrum,...,4,16,32,63,90478,90478,3.066872,3.066872,False,16
5,5,True,0,2019-03-07 10:27:00,90004,66126,5,49.433693,4,Doku-Zentrum,...,7,10,27,66,90478,90478,3.066872,3.066872,False,10


In [5]:
def get_trip_counts_by_hour(selected_hour, bike_data):
    # make a DataFrame with locations for each bike station
    locations = bike_data.groupby("p_uid_start").first()
    locations = locations.loc[:, ["Latitude_start",
                                 "Longitude_start",
                                 "Place_start"]]
    
    #select one time of day
    subset = bike_data # bike_data["hour"]] ==selected_hour]
    
    # count trips for each destination
    departure_counts =  subset.groupby("p_uid_start").count()
    departure_counts = departure_counts.iloc[:,[0]]
    departure_counts.columns= ["Departure Count"]
    
    # count trips for each origin
    arrival_counts =  subset.groupby("p_uid_end").count().iloc[:,[0]]
    arrival_counts.columns= ["Arrival Count"]

    #join departure counts, arrival counts, and locations
    trip_counts = departure_counts.join(locations).join(arrival_counts)
    return trip_counts

# print a sample to check our code works
# get_trip_counts_by_hour().head()

In [6]:
def plot_station_counts(trip_counts):
    # generate a new map
    folium_map = folium.Map(location=[49.452633, 11.095934],
                            zoom_start=13,
                            tiles="CartoDB dark_matter",
                            width='50%')

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["Departure Count"]-row["Arrival Count"])
        
        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["Place_start"],
                          row["Arrival Count"],
                          row["Departure Count"],
                          net_departures)
        
        # radius of circles
        radius = net_departures/20
        
        # choose the color of the marker
        if net_departures>0:
            # color="#FFCE00" # orange
            # color="#007849" # green
            color="#E37222" # tangerine
        else:
            # color="#0375B4" # blue
            # color="#FFCE00" # yellow            
            color="#0A8A9F" # teal
        
        # add marker to the map
        folium.CircleMarker(location=(row["Latitude_start"],
                                      row["Longitude_start"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map

In [7]:
# plot net departures at 9AM

trip_counts = get_trip_counts_by_hour()
plot_station_counts(trip_counts)

TypeError: get_trip_counts_by_hour() missing 2 required positional arguments: 'selected_hour' and 'bike_data'

In [ ]:
# plot net departures at 6PM

trip_counts = get_trip_counts_by_hour(3)
folium_map = plot_station_counts(trip_counts)
folium_map

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters

In [ ]:
def get_kernel(kernel_size, blur=1/20, halo=.001):
    """
    Create an (n*2+1)x(n*2+1) numpy array.
    Output can be used as the kernel for convolution.
    """
    
    # generate x and y grids
    x, y = np.mgrid[0:kernel_size*2+1, 0:kernel_size*2+1]
    
    center = kernel_size + 1  # center pixel
    r = np.sqrt((x - center)**2 + (y - center)**2)  # distance from center
    
    # now compute the kernel. This function is a bit arbitrary. 
    # adjust this to get the effect you want.
    kernel = np.exp(-r/kernel_size/blur) + (1 - r/r[center,0]).clip(0)*halo
    return kernel

def add_lines(image_array, xys, width=1, weights=None):
    """
    Add a set of lines (xys) to an existing image_array
    width: width of lines
    weights: [], optional list of multipliers for lines. 
    """
    
    for i, xy in enumerate(xys):  # loop over lines
        # create a new gray scale image
        image = Image.new("L",(image_array.shape[1], image_array.shape[0]))
        
        # draw the line
        ImageDraw.Draw(image).line(xy, 200, width=width)
        
        #convert to array
        new_image_array = np.asarray(image, dtype=np.uint8).astype(float)
        
        # apply weights if provided
        if weights is not None:
            new_image_array *= weights[i]
            
        # add to existing array
        image_array += new_image_array

    # convolve image
    new_image_array = scipy.ndimage.filters.convolve(image_array, get_kernel(width*4)) 
    return new_image_array

In [ ]:
def to_image(array, hue=.62):
    """converts an array of floats to an array of RGB values using a colormap"""
    
    # apply saturation function
    image_data = np.log(array + 1)
    
    # create colormap, change these values to adjust to look of your plot
    saturation_values = [[0, 0], [1, .68], [.78, .87], [0, 1]]
    colors = [hsv_to_rgb([hue, x, y]) for x, y in saturation_values]
    cmap = LinearSegmentedColormap.from_list("my_colormap", colors)
    
    # apply colormap
    out = cmap(image_data/image_data.max())
    
    # convert to 8-bit unsigned integer
    out = (out*255).astype(np.uint8)
    return out

In [ ]:
xys = [(10,10,200,200), (175,10,175,200), (200,10,10,200)]  # some lines
weights = np.array([ 2,1,.7])/100  # some weights

# create the image_data
new_image_data = add_lines(np.zeros((220,220)),
                           xys,
                           width=4,
                           weights=weights) 

# show the image
Image.fromarray(to_image(new_image_data),mode="RGBA")

In [ ]:
min_lat = bike_data["Latitude_start"].min()
max_lat = bike_data["Latitude_start"].max()
max_lon = bike_data["Longitude_start"].max()
min_lon = bike_data["Longitude_start"].min()

def latlon_to_pixel(lat, lon, image_shape):
    # longitude to pixel conversion (fit data to image)
    delta_x = image_shape[1]/(max_lon-min_lon)
    
    # latitude to pixel conversion (maintain aspect ratio)
    delta_y = delta_x/np.cos(lat/360*np.pi*2)
    pixel_y = (max_lat-lat)*delta_y
    pixel_x = (lon-min_lon)*delta_x
    return (pixel_y,pixel_x)


def row_to_pixel(row,image_shape):
    """
    convert a row (1 trip) to pixel coordinates
    of start and end point
    """
    start_y, start_x = latlon_to_pixel(row["Latitude_start"], 
                                       row["Longitude_start"], image_shape)
    end_y, end_x = latlon_to_pixel(row["Latitude_end"], 
                                   row["Longitude_end"], image_shape)
    xy = (start_x, start_y, end_x, end_y)
    return xy

In [ ]:
paths = bike_data[bike_data.hour==8]
paths = paths.iloc[:3000,:]

# generate empty pixel array, choose your resolution
image_data = np.zeros((1800,1800))

# generate pixel coordinates of starting points and end points
xys = [row_to_pixel(row, image_data.shape) for i, row in paths.iterrows()]

# draw the lines
image_data = add_lines(image_data, xys, weights=None, width = 1)
Image.fromarray(to_image(image_data*10)[:,:,:3],mode="RGB")

In [ ]:

# make a list of locations (latitude longitude) for each station id
locations = bike_data.groupby("p_uid_start").mean()
locations = locations.loc[:,["Latitude_start", "Longitude_start"]]

# group by each unique pair of (start-station, end-station) and count the number of trips
bike_data["path_id"] = [(id1,id2) for id1,id2 in zip(bike_data["p_uid_start"], 
                                                     bike_data["p_uid_end"])]
paths = bike_data[bike_data["hour"]==9].groupby("path_id").count().iloc[:,[1]] 
paths.columns = ["Trip Count"]

# select only paths with more than X trips
paths = paths[paths["Trip Count"]>1]
paths["p_uid_start"] = paths.index.map(lambda x:x[0])
paths["p_uid_end"] = paths.index.map(lambda x:x[1])
paths = paths[paths["p_uid_start"]!=paths["p_uid_end"]]

# join latitude/longitude into new table
paths = paths.join(locations,on="p_uid_start")
locations.columns = ["Latitude_end", "Longitude_end"]
paths = paths.join(locations,on="p_uid_end")
paths.index = range(len(paths))

paths.shape

In [ ]:
def get_image_data(paths, min_count=0, max_count=None):
    # generate empty pixel array
    image_data = np.zeros((900*2,400*2))
    
    # generate pixel coordinates of starting points and end points
    if max_count is None:
        max_count = paths["Trip Count"].max()+1
    selector = (paths["Trip Count"]>= min_count) & (paths["Trip Count"]< max_count)
    xys = [row_to_pixel(row, image_data.shape) for i, row in paths[selector].iterrows()]

    # draw the lines
    image_data = add_lines(image_data, xys, weights=paths["Trip Count"], width = 1)
    return image_data

In [ ]:
Image.fromarray(to_image(get_image_data(paths)*10)[:,:,:3],mode="RGB")

In [8]:
def add_alpha(image_data):
    """
    Uses the Value in HSV as an alpha channel. 
    This creates an image that blends nicely with a black background.
    """
    
    # get hsv image
    hsv = rgb_to_hsv(image_data[:,:,:3].astype(float)/255)
    
    # create new image and set alpha channel
    new_image_data = np.zeros(image_data.shape)
    new_image_data[:,:,3] = hsv[:,:,2]
    
    # set value of hsv image to either 0 or 1.
    hsv[:,:,2] = np.where(hsv[:,:,2]>0, 1, 0)
    
    # combine alpha and new rgb
    new_image_data[:,:,:3] = hsv_to_rgb(hsv)
    return new_image_data

In [9]:
# create the map
folium_map = folium.Map(location=[49.452030, 11.076750],
                        zoom_start=11,
                        tiles="CartoDB dark_matter",
                        width='100%')

# create the overlay
map_overlay = add_alpha(to_image(image_data*10))

# compute extent of image in lat/lon
aspect_ratio = map_overlay.shape[1]/map_overlay.shape[0]
delta_lat = (max_lon-min_lon)/aspect_ratio*np.cos(min_lat/360*2*np.pi)

# add the image to the map
img = folium.raster_layers.ImageOverlay(map_overlay,
                           bounds = [(max_lat-delta_lat,min_lon),(max_lat,max_lon)],
                           opacity = 1,
                           name = "Paths")

img.add_to(folium_map)
folium.LayerControl().add_to(folium_map)

# show the map
folium_map

NameError: name 'to_image' is not defined

In [10]:
folium_map = folium.Map(location=[49.452030, 11.076750],
                        zoom_start=13,
                        tiles="CartoDB dark_matter",
                        width='100%')

thresholds = [5,15,25]

for i,t in enumerate(thresholds):
    upper = thresholds[i+1] if i<len(thresholds)-1 else None
    image_data = get_image_data(paths, t, upper)
    name = "{} < Num. Trips < {}".format(t,"max" if upper is None else upper)

    map_overlay = add_alpha(to_image(image_data*10))
    delta_lat = (max_lon-min_lon)/map_overlay.shape[1]*map_overlay.shape[0]*np.cos(min_lat/360*2*np.pi)
    img = folium.raster_layers.ImageOverlay(map_overlay,
                               bounds = [(max_lat-delta_lat,min_lon),(max_lat,max_lon)],
                               name = name)

    img.add_to(folium_map)

lc = folium.LayerControl().add_to(folium_map)

folium_map

NameError: name 'get_image_data' is not defined

In [57]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters
import time
import datetime
import os.path
import io

import os
os.environ["PATH"] += os.pathsep + "."

%matplotlib inline

In [65]:
def interpolate(df1, df2, x):
    """return a weighted average of two dataframes"""
    df = df1 * (1 - x) + df2 * x
    return df.replace(np.nan, 0)


def get_trip_counts_by_minute(float_hour, data):
    """get an interpolated dataframe for any time, based
    on hourly data"""
    
    columns = ["Latitude_start",
               "Longitude_start",
               "Departure Count",
               "Arrival Count"]
    df1 = get_trip_counts_by_hour(int(float_hour), data) # ausgelagerte Funktion
    df2 = get_trip_counts_by_hour(int(float_hour) + 1, data)
    
    df = interpolate(df1.loc[:,columns], 
                     df2.loc[:,columns], 
                     float_hour % 1)
    
    df["p_uid_start"] = df1["p_uid_start"]
    return df

In [70]:
df1

NameError: name 'df1' is not defined

In [66]:
plot_station_counts(get_trip_counts_by_minute(9.5, bike_data), zoom_start=14) # ausgelagerte Funktion

KeyError: 'p_uid_start'

In [44]:
import pandas as pd
import numpy as np
import folium
from PIL import Image, ImageDraw, ImageFont
from folium import plugins
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters
import io
import os



def get_kernel(kernel_size, blur=1 / 20, halo=.001):
    """
    Create an (n*2+1)x(n*2+1) numpy array.
    Output can be used as the kernel for convolution.
    """

    # generate x and y grids
    x, y = np.mgrid[0:kernel_size * 2 + 1, 0:kernel_size * 2 + 1]

    center = kernel_size + 1  # center pixel
    r = np.sqrt((x - center) ** 2 + (y - center) ** 2)  # distance from center

    # now compute the kernel. This function is a bit arbitrary.
    # adjust this to get the effect you want.
    kernel = np.exp(-r / kernel_size / blur) + (1 - r / r[center, 0]).clip(0) * halo
    return kernel


def add_lines(image_array, xys, width=1, weights=None):
    """
    Add a set of lines (xys) to an existing image_array
    width: width of lines
    weights: [], optional list of multipliers for lines.
    """

    for i, xy in enumerate(xys):  # loop over lines
        # create a new gray scale image
        image = Image.new("L", (image_array.shape[1], image_array.shape[0]))

        # draw the line
        ImageDraw.Draw(image).line(xy, 200, width=width)

        # convert to array
        new_image_array = np.asarray(image, dtype=np.uint8).astype(float)

        # apply weights if provided
        if weights is not None:
            new_image_array *= weights[i]

        # add to existing array
        image_array += new_image_array

    # convolve image
    new_image_array = scipy.ndimage.filters.convolve(image_array, get_kernel(width * 4))
    return new_image_array


def to_image(array, hue=.62):
    """converts an array of floats to an array of RGB values using a colormap"""

    # apply saturation function
    image_data = np.log(array + 1)

    # create colormap, change these values to adjust to look of your plot
    saturation_values = [[0, 0], [.75, .68], [.78, .87], [0, 1]]
    colors = [hsv_to_rgb([hue, x, y]) for x, y in saturation_values]
    cmap = LinearSegmentedColormap.from_list("my_colormap", colors)

    # apply colormap
    out = cmap(image_data / image_data.max())

    # convert to 8-bit unsigned integer
    out = (out * 255).astype(np.uint8)
    return out


def get_min_max(bike_data):
    min_lat = bike_data["Latitude_start"].min()
    max_lat = bike_data["Latitude_start"].max()
    max_lon = bike_data["Longitude_start"].max()
    min_lon = bike_data["Longitude_start"].min()
    return min_lat, max_lat, min_lon, max_lon


def latlon_to_pixel(lat, lon, image_shape, bounds):
    min_lat, max_lat, min_lon, max_lon = bounds

    # longitude to pixel conversion (fit data to image)
    delta_x = image_shape[1] / (max_lon - min_lon)

    # latitude to pixel conversion (maintain aspect ratio)
    delta_y = delta_x / np.cos(lat / 360 * np.pi * 2)
    pixel_y = (max_lat - lat) * delta_y
    pixel_x = (lon - min_lon) * delta_x
    return (pixel_y, pixel_x)


def row_to_pixel(row,image_shape, columns=None):
    """
    convert a row (1 trip) to pixel coordinates
    of start and end point
    """
    start_y, start_x = latlon_to_pixel(row["Latitude_start"],
                                       row["Longitude_start"], image_shape)
    end_y, end_x = latlon_to_pixel(row["Latitude_end"],
                                   row["Longitude_end"], image_shape)
    xy = (start_x, start_y, end_x, end_y)
    return xy



def start_interactive_maps():
    bike_data = pd.read_csv("D:\Eigene Dateien\OneDrive\Dokumente\GitHub\PDS_Project/nextbike/data/output/Trips.csv")
    bike_data["Start_Time"] = pd.to_datetime(bike_data["Start_Time"])
    bike_data["End_Time"] = pd.to_datetime(bike_data["End_Time"])
    # TODO: to be removed, if work
    # bike_data["hour"] = bike_data["Start_Time"].map(lambda x: x.hour)
    return bike_data


def plot_station_counts(trip_counts, zoom_start=13):
    # generate a new map
    folium_map = folium.Map(location=[49.452633, 11.095934],
                            zoom_start=zoom_start,
                            tiles="CartoDB dark_matter",
                            width="100%")

    # for each row in the data, add a circle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["Departure Count"] - row["Arrival Count"])

        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["Place_start"],
                                       row["Arrival Count"],
                                       row["Departure Count"],
                                       net_departures)

        # radius of circles
        radius = net_departures / 10

        # choose the color of the marker
        if net_departures > 0:
            # color="#FFCE00" # orange
            # color="#007849" # green
            color = "#E37222"  # tangerine
        else:
            # color="#0375B4" # blue
            # color="#FFCE00" # yellow
            color = "#0A8A9F"  # teal

        # add marker to the map
        folium.CircleMarker(location=(row["Latitude_start"],
                                      row["Longitude_start"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map


def get_locations(bike_data):
    locations = bike_data.groupby("p_uid_start").first()
    locations = locations.loc[:, ["Latitude_start",
                                 "Longitude_start",
                                 "Place_start"]]
    return locations


def get_trip_counts_by_hour(selected_hour, bike_data):
    # make a DataFrame with locations for each bike station
    locations = get_locations(bike_data)

    # select one time of day
    subset = bike_data[bike_data["Hour_start"] == selected_hour]

    # count trips for each destination
    departure_counts = subset.groupby("p_uid_start").count()
    departure_counts = departure_counts.iloc[:, [0]]
    departure_counts.columns = ["Departure Count"]

    # count trips for each origin
    arrival_counts = subset.groupby("p_uid_end").count().iloc[:, [0]]
    arrival_counts.columns = ["Arrival Count"]

    # join departure counts, arrival counts, and locations
    trip_counts = departure_counts.join(locations).join(arrival_counts)
    # print(trip_counts.head(5))
    return trip_counts

# print a sample to check our code works
# get_trip_counts_by_hour().head()


def add_alpha(image_data):
    """
    Uses the Value in HSV as an alpha channel.
    This creates an image that blends nicely with a black background.
    """

    # get hsv image
    hsv = rgb_to_hsv(image_data[:, :, :3].astype(float) / 255)

    # create new image and set alpha channel
    new_image_data = np.zeros(image_data.shape)
    new_image_data[:, :, 3] = hsv[:, :, 2]

    # set value of hsv image to either 0 or 1.
    hsv[:, :, 2] = np.where(hsv[:, :, 2] > 0, 1, 0)

    # combine alpha and new rgb
    new_image_data[:, :, :3] = hsv_to_rgb(hsv)
    return new_image_data


def create_image_map(image_data, bounds):
    min_lat, max_lat, min_lon, max_lon = bounds
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=13,
                            tiles="CartoDB dark_matter",
                            width='100%')

    # create the overlay
    map_overlay = add_alpha(to_image(image_data))

    # compute extent of image in lat/lon
    aspect_ratio = map_overlay.shape[1]/map_overlay.shape[0]
    delta_lat = (max_lon-min_lon)/aspect_ratio*np.cos(min_lat/360*2*np.pi)

    # add the image to the map
    img = folium.raster_layers.ImageOverlay(map_overlay,
                                            bounds=[(max_lat-delta_lat, min_lon), (max_lat, max_lon)],
                                            opacity=1,
                                            name="Paths")

    img.add_to(folium_map)
    folium.LayerControl().add_to(folium_map)

    # return the map
    return folium_map


def interpolate(df1, df2, x):
    """return a weighted average of two dataframes"""
    df = df1 * (1 - x) + df2 * x
    return df.replace(np.nan, 0)


def get_trip_counts_by_minute(float_hour, data):
    """get an interpolated dataframe for any time, based
    on hourly data"""

    columns = ["Latitude_start",
               "Longitude_start",
               "Departure Count",
               "Arrival Count"]
    df1 = get_trip_counts_by_hour(int(float_hour), data)
    df2 = get_trip_counts_by_hour(int(float_hour) + 1, data)

    df = interpolate(df1.loc[:, columns],
                     df2.loc[:, columns],
                     float_hour % 1)
    # print("PRINT DF1")
    # print(df1.head())
    # print("PRINT DF:")
    # print(df.head())
    # df["p_uid_start"] = df1["p_uid_start"]
    return df

#TODO: SAvePath rechecking
def go_arrivals_frame(i, hour_of_day, save_path):
    # create the map object
    data = get_trip_counts_by_minute(hour_of_day, bike_data)
    my_frame = plot_station_counts(data, zoom_start=14)

    # generate the png file as a byte array
    png = my_frame._to_png()

    #  now add a caption to the image to indicate the time-of-day.
    hour = int(hour_of_day)
    minutes = int((hour_of_day % 1) * 60)

    # create a PIL image object
    image = Image.open(io.BytesIO(png))
    draw = ImageDraw.ImageDraw(image)

    # load a font
    font = ImageFont.truetype("Roboto-Light.ttf", 30)

    # draw time of day text
    draw.text((20, image.height - 50),
              "time: {:0>2}:{:0>2}h".format(hour, minutes),
              fill=(255, 255, 255),
              font=font)

    # draw title
    draw.text((image.width - 400, 20),
              "Net Arrivals vs Time of Day",
              fill=(255, 255, 255),
              font=font)

    # write to a png file
    # io.output.save_fig(image, "frame_{:0>5}.png")
    filename = os.path.join(save_path, "frame_{:0>5}.png".format(i))
    image.save(filename, "PNG")
    return image



bike_data = start_interactive_maps()
plot_station_counts(get_trip_counts_by_minute(9.5, bike_data), zoom_start=14)

dir_name = 'frames'
if not os.path.exists(dir_name):
    os.mkdir(dir_name)
go_arrivals_frame(1, 8.5)

arrival_times = np.arange(6, 23, .2)
for i, hour in enumerate(arrival_times):
    go_arrivals_frame(i, hour, "frames")

KeyError: 'Place_start'

In [38]:
df1

NameError: name 'df1' is not defined

In [33]:
locations = bike_data.groupby("p_uid_start").first()


In [34]:
locations

,index,p_spot_start,p_place_type_start,Start_Time,Bike_Number,p_bikes_start,Latitude_start,b_bike_type_start,Place_start,Start_Place_id,...,Month_end,Day_end,Hour_end,Minute_end,Day_of_year_end,Postalcode_start,Postalcode_end,Dist_start,Dist_end,Direction
p_uid_start,,,,,,,,,,,,,,,,,,,,,
66093,4424,True,0,2019-04-11 11:04:00,90206,2,49.452633,4,Ohm-Hochschule (Wassertorstr.),4602.0,...,4,11,11,4,101,90489,90478,1.187062,3.082876,False
66096,4263,True,0,2019-03-31 19:06:00,90169,5,49.453323,4,Heilig-Geist-Spital (Spitalgasse),4605.0,...,3,31,19,6,90,90403,90478,0.135309,1.764837,False
66097,3658,True,0,2019-02-16 15:50:00,90101,5,49.451032,4,Vordere Ledergasse / Schlotfegergasse,4606.0,...,2,16,16,50,47,90402,90478,0.731215,1.776586,False
66100,4157,True,0,2019-09-20 14:23:00,90019,2,49.459501,71,St. Johannis,4608.0,...,9,20,14,23,263,90419,90478,1.618017,3.261340,False
66101,4703,True,0,2019-02-12 15:56:00,90101,4,49.458798,4,Maxtor,4609.0,...,2,12,16,56,43,90408,90478,0.742493,1.776586,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26517674,155365,False,12,2019-12-31 23:38:00,90170,1,49.458187,71,BIKE 90170,0.0,...,12,31,23,38,365,90403,90409,0.680981,1.202475,False
26518511,75919,False,12,2019-12-31 23:01:00,90007,1,49.461427,71,BIKE 90007,0.0,...,12,31,23,1,365,90419,90403,1.922671,0.671340,True
26519150,17575,False,12,2019-12-31 23:49:00,90207,1,49.443662,71,BIKE 90207,0.0,...,12,31,23,49,365,90443,90443,1.644100,1.608058,True


In [35]:
locations1 = locations.loc[:, ["Latitude_start",
                             "Longitude_start",
                             "Place_start"]]

In [36]:
locations1

,Latitude_start,Longitude_start,Place_start
p_uid_start,,,
66093,49.452633,11.095934,Ohm-Hochschule (Wassertorstr.)
66096,49.453323,11.078821,Heilig-Geist-Spital (Spitalgasse)
66097,49.451032,11.069654,Vordere Ledergasse / Schlotfegergasse
66100,49.459501,11.060263,St. Johannis
66101,49.458798,11.081235,Maxtor
...,...,...,...
26517674,49.458187,11.077534,BIKE 90170
26518511,49.461427,11.057139,BIKE 90007
26519150,49.443662,11.061077,BIKE 90207
